In [1]:
%%capture
!pip install git+https://github.com/openai/CLIP.git transformers gdown wandb katna more_itertools
!pip install git+https://github.com/cene555/ru-clip-tiny.git 
!gdown -O ru-clip-tiny.pkl https://drive.google.com/uc?id=1-3g3J90pZmHo9jbBzsEmr7ei5zm3VXOL

# эмбеддинги видео
!gdown 1eCSqwe9ByUkc-hA108OrTxnXSdEOR5TZ

In [4]:
import os
import gc
import io
import os
import sys
import cv2
import clip
import json
import wandb
import pickle
import random
import argparse
import warnings
import numpy as np
import pandas as pd
import more_itertools
from time import time
import matplotlib.pyplot as plt
from IPython.display import Image 
from dataclasses import dataclass, field
from typing import Tuple, Optional, Union

import PIL
import PIL.Image
from PIL import Image

from tqdm import tqdm, trange
from tqdm.contrib import tzip
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as nnf
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader
from torch.utils.checkpoint import checkpoint_sequential

from torchvision import transforms
import torchvision.transforms as T
import torchvision.transforms.functional as TF


import transformers
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, AdamW, get_linear_schedule_with_warmup
from transformers.optimization import Adafactor, AdafactorSchedule


warnings.filterwarnings("ignore", category=UserWarning)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# https://github.com/cene555/ru-clip-tiny.git

class Predictor:
    def __init__(self):
        self.tokenizer = Tokenizer()
        self.transform = get_transform()
        
        self.model = RuCLIPtiny()
        self.model.load_state_dict(torch.load('/kaggle/working/ru-clip-tiny.pkl', map_location=device))
        self.model = self.model.to(device).eval()
        for x in self.model.parameters(): x.requires_grad = False
        torch.cuda.empty_cache()

    def prepare_images_features(self, images, device='cpu'):
        images_features = []
        for image in images:
            image = self.transform(image)
            with torch.no_grad():
                image_features = self.model.encode_image(image.unsqueeze(0).to(device)).float().cpu()[0]
            images_features.append(image_features)
        images_features = torch.stack(images_features, axis=0)
        images_features /= images_features.norm(dim=-1, keepdim=True)
        return images_features.cpu()

    def prepare_text_features(self,texts, max_len=77, device='cpu'):
        texts_features = []
        for text in texts:
            tokens = self.tokenizer.tokenize([text], max_len)
            with torch.no_grad():
                text_features = self.model.encode_text(tokens[0].to(device), tokens[1].to(device)).float().cpu()[0]
            texts_features.append(text_features)
        texts_features = torch.stack(texts_features, axis=0)
        texts_features /= texts_features.norm(dim=-1, keepdim=True)
        return texts_features

    def __call__(self,images, classes, max_len=77, device='cpu'):
        self.model.eval().to(device)
        image_features = self.prepare_images_features(images, device)
        texts_features = self.prepare_text_features(classes, max_len, device)
        return 1 * image_features @ texts_features.T

In [ ]:
import torch
import torchvision.transforms as transforms
import numpy as np
import clip
from Katna.video import Video
from PIL import Image
import cv2

def read_video(path, frames_num=9, window=30, resize=True):
    frames = []
    cap = cv2.VideoCapture(path)
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    N = length // frames_num
    
    current_frame = 1
    for i in range(length):
        ret, frame = cap.read(current_frame)
        if ret and i==current_frame and len(frames)<frames_num:
            frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if resize:
                size = 193, 193
                frame.thumbnail(size, Image.ANTIALIAS)
            
            frames.append(frame)
            current_frame += N
        
    cap.release()
    return frames


class FrameExtractor:
    def __init__(self, num_key_frames=20, num_frames=10, use_keyframe=False):
        self.N_keyframe = num_key_frames
        self.N_frames = num_frames
        self.use_keyframe = use_keyframe
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.predictor = Predictor()
        self.vid_obj = Video()

    def extract_video_keyframes(self,file_path):

        video_duration = self.vid_obj._get_video_duration_with_cv(file_path)
        video_split_threshold_in_minutes = 20
        if video_duration > (video_split_threshold_in_minutes * 60):
            top_frames = self.vid_obj.extract_video_keyframes_big_video(self.N_keyframe, file_path)
        else:
            top_frames = self.vid_obj._extract_keyframes_from_video(self.N_keyframe, file_path)
        return top_frames

    def fix_color(self, frame):
        if frame.ndim == 3 and frame.shape[2] == 4:  # Convert BGRA frames to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGB)
        elif frame.ndim == 3 and frame.shape[2] == 1:  # Convert grayscale frames to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
        else:  # Convert BGR frames to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame
    
    def get_frames(self, vid_path, description, scores=False, use_keyframe=False):
        if use_keyframe:
            keyframes = self.extract_video_keyframes(vid_path)
            keyframes = [self.fix_color(frame) for frame in keyframes]
        else:
            keyframes = read_video(path=vid_path, frames_num=self.N_keyframe, resize=False)
        
        text_probs = self.predictor(images=keyframes,
                       classes=[description],
                       max_len=77, device=device)

        keyframe_scores = text_probs.cpu().numpy().flatten().tolist()
            
        data = []
        for frame, score in zip(keyframes, keyframe_scores):
            data.append([frame, score])

        data = sorted(data, key=lambda x: x[1], reverse=True)
        frames = data[:self.N_frames] #(frame, score)
        if scores:
            return frames
        res = []
        for frame_with_score in frames:
            frame = frame_with_score[0]
            size = 193, 193
            if use_keyframe:
                frame = Image.fromarray(frame)
            frame.thumbnail(size, Image.ANTIALIAS)
            res.append(frame)
        return res

In [ ]:
def image_grid(imgs, rows, cols):
    pils = imgs
    
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid        
        
    cap.release()
    return frames

In [ ]:
out_path = f"Features_train_full_ru_2_2.pkl"
df_train = pd.read_csv('/kaggle/input/csv-nto-final/updatedtrain.csv')

all_embeddings = []
all_captions = []
i = 0
frame_extractor = FrameExtractor(num_key_frames=12, num_frames=4)

for video_name, question, answer in tzip(df_train.video_name, df_train.question, df_train.answer):
    name = f'{video_path}/{video_name}.mp4'
    text = f'Q: {question} A: {answer}'
    
    if os.path.exists(name):
        frames = frame_extractor.get_frames(name, question, scores=False, use_keyframe=False)
        if len(frames) == 4:
            image = image_grid(frames, 2, 2)
            with torch.no_grad():
                image = preprocess(image).unsqueeze(0).to(device)
                prefix = clip_model.encode_image(image).cpu()
            all_embeddings.append(prefix)
            all_captions.append(text)
    
with open(out_path, 'wb') as f:
    pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

print('Done')
print("%0d embeddings saved " % len(all_embeddings))

In [6]:
class ClipCocoDataset(Dataset):
    
    def __init__(self, data_path: str,  prefix_length= 50, gpt2_type = "sberbank-ai/rugpt3large_based_on_gpt2",
                 normalize_prefix=False):
        self.tokenizer = AutoTokenizer.from_pretrained(gpt2_type)
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        with open(data_path, 'rb') as f:
            all_data = pickle.load(f)
        print("Data size is %0d" % len(all_data["clip_embedding"]))
        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        captions_raw = all_data["captions"]
        
        self.captions = captions_raw
        
        
        self.captions_tokens = []
        self.caption2embedding = []
        max_seq_len = 0
        i=0
        for caption in tqdm(captions_raw):
                self.captions_tokens.append(torch.tensor(self.tokenizer.encode(caption), dtype=torch.int64))
                self.caption2embedding.append(self.prefixes[i])
                i+=1
                max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])    
    
        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))

    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            self.captions_tokens[item] = tokens
        mask = tokens.ge(0)
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask
    
    def __len__(self) -> int:
        return len(self.captions_tokens)

    def __getitem__(self, item):
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[item]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        return tokens, mask, prefix

In [7]:
class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    
def freeze(
    model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=True,
):
    
    for name, p in model.named_parameters(): 
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return model

class ClipCaptionModel(nn.Module):
    def __init__(self, backbone, prefix_length: int, prefix_size: int = 768):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = AutoModelForCausalLM.from_pretrained(backbone)
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                  self.gpt_embedding_size * prefix_length))

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)
    
    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):

        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)

        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

  
class ClipCaptionPrefix(ClipCaptionModel):
    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()
    
    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

In [8]:
def filter_ngrams(output_text):
    a_pos = output_text.find(' A:')
    sec_a_pos = output_text.find(' A:', a_pos + 1)
    return output_text[:sec_a_pos]


def generate2(
        model,
        tokenizer,
        prompt='',
        embed=None,
        entry_count=1,
        entry_length=50,  # maximum number of words
        top_p=0.98,
        temperature=1.,
        stop_token='<|endoftext|>',
):
    tokens = None
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in range(entry_count):
            if not tokens:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)

            emb_tokens = model.gpt.transformer.wte(tokens)

            if embed[entry_idx] is not None:
                emb = embed[entry_idx].unsqueeze(0).to(device)
                generated = torch.cat((emb, emb_tokens), dim=1)
            else:
                generated = emb_tokens

            for i in range(entry_length):
                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(nnf.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                #
                top_k = 2000
                top_p = 0.98
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)

                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)

                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())

            output_text = tokenizer.decode(output_list)
            output_text = filter_ngrams(output_text)
            generated_list.append(output_text)

    return generated_list

In [9]:
import nltk

def get_score(gt, pred):
    score = nltk.translate.bleu_score.sentence_bleu([gt.lower().replace('<|endoftext|>','').replace("!", "").split()], pred.lower().replace('<|endoftext|','').split(), weights = (0.5, 0.5))
    return score

In [10]:
def train(dataset, model: ClipCaptionModel, args,
          warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):

    device = torch.device('cuda')
    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    model = model.to(device)
    
    model = freeze(model)
    model.train()
    optimizer = AdamW(model.parameters(), lr=args.lr,betas=(0.9, 0.995))

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs * len(train_dataloader)
    )
    current_loss = 1000000

    for epoch in range(epochs):
        print(f">>> Training epoch {epoch}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        step=0
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            step+=1
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
            
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, dataset.prefix_length - 1: -1]

            loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)

            segments = 2
            
            loss.backward()
            
            optimizer.step()
            scheduler.step()
            progress.set_postfix({"loss": loss.item()})
            
            clipping_value = 0.5 
            optimizer.step()
            scheduler.step()
            
            wandb.log({"loss":  loss.item()})
            
            progress.update()
            
            del tokens
            del mask
            del prefix
            torch.clear_autocast_cache()
            torch.cuda.empty_cache()
            
            if (idx + 1) % 7000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )
        
        step = 0
        val_loss = []
        step = 0
        scores = []
        val_loss = []
        for idx, (tokens, mask, prefix) in enumerate(val_loader):
            with torch.no_grad():
                step += 1

                tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)

                outputs = model(tokens, prefix, mask)
                logits = outputs.logits[:, dataset.prefix_length - 1: -1]
                loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)
                val_loss.append(loss.item())

                progress.set_postfix({"val_loss": loss.item()})
                wandb.log({"val_loss": loss.item()})

                progress.update()

                del tokens
                del mask
                del prefix
                

        torch.clear_autocast_cache()
        torch.cuda.empty_cache()

        progress.close()
        
        if epoch in [4, 5, 6]:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}_{epoch+1}.pt"))   

        val_loss = sum(val_loss) / len(val_loss)
        if val_loss < current_loss:
            print(f'[+] Val loss has increased from {current_loss:.4f} to {val_loss:.4f}')
        else:            
            print(f'[-] Val loss has not improved. {val_loss:.4f}')
        progress.close()
    return model 

In [11]:
class Args():
    def __init__(self):
        self.backbone = 'sberbank-ai/rugpt3small_based_on_gpt2'
        self.data = '/kaggle/working/Features_train_full_ru_2_2.pkl'
        self.out_dir = 'checkpoints_larger'
        self.wandb_key = 'YOUR WANDB KEY HERE'
        self.project_name = "final-nto"
        self.prefix = 'prefix_2_2_small'
        self.epochs = 7
        self.save_every = 1
        self.prefix_length = 50
        self.batch_size = 16
        self.bs = 16
        self.debug = True
        self.only_prefix = False
        self.lr = 5e-5
        self.train_size = 0.9
        
args = Args()

In [ ]:
import wandb

if args.debug:
    wandb.login(key=args.wandb_key)
    wandb.init(project=args.project_name, config=args)

In [13]:
wandb.config = {
    "learning_rate": args.lr,
    "epochs": args.epochs,
    "batch_size": args.bs
}
prefix_length = args.prefix_length
dataset = ClipCocoDataset(args.data, prefix_length)
model = ClipCaptionModel(backbone = args.backbone, prefix_length = 50) 
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

print("Train both prefix and GPT")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Data size is 25228


100%|██████████| 25228/25228 [00:03<00:00, 7482.41it/s]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Train both prefix and GPT


In [14]:
train_size = round(args.train_size * len(dataset))
val_size = len(dataset) - train_size
print(f"Train size: {train_size}, Val size: {val_size}")

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)

Train size: 22705, Val size: 2523


In [15]:
sys.stdout.flush()
train(dataset, model, args, output_dir=args.out_dir, output_prefix=args.prefix)

>>> Training epoch 0


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
prefix_2_2_small: 1733it [06:25,  4.50it/s, val_loss=2.7]                          

[+] Val loss has increased from 1000000.0000 to 2.8236
>>> Training epoch 1



prefix_2_2_small: 1733it [06:23,  4.52it/s, val_loss=2.52]                          

[+] Val loss has increased from 1000000.0000 to 2.3086
>>> Training epoch 2


...

prefix_2_2_small: 1733it [06:22,  4.53it/s, val_loss=2.08]                          


[+] Val loss has increased from 1000000.0000 to 2.2318
>>> Training epoch 6


prefix_2_2_small: 1733it [06:23,  4.52it/s, val_loss=1.94]                          


[+] Val loss has increased from 1000000.0000 to 2.2282


ClipCaptionModel(
  (gpt): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50264, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
       